## **silver-companies-cash-flow-statement**
<p>This notebook retrieves raw cash flow statements for listed companies, including key metrics like operating, investing, and financing cash flows. It stores the data in its original form to enable financial analysis and trend identification at later stages.</p>

#### **Importing necessary libraries**

In [6]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, 763ba16a-a488-49ce-96cd-40c08b0e58e2, 10, Finished, Available, Finished)

#### **Variables**

In [7]:
file_name = "comp_cash_flow_data"
layer_name = "bronze"

StatementMeta(, 763ba16a-a488-49ce-96cd-40c08b0e58e2, 11, Finished, Available, Finished)

#### **Step 2: Read the Balance Sheet and Breakdown Categories Data**

In [8]:
csh_flow_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

breakdown_categories = spark.read.csv("Files/master_files/breakdown_categories.csv", header=True)

StatementMeta(, 763ba16a-a488-49ce-96cd-40c08b0e58e2, 12, Finished, Available, Finished)

#### **Step 3: Filters the Unnecessary Breakdown Categories from the Balance Sheet**

In [9]:
output_df = csh_flow_df\
                    .join(broadcast(breakdown_categories), csh_flow_df.Breakdown == breakdown_categories['Account Name'], 'inner')\
                    .select(csh_flow_df.Breakdown, csh_flow_df.Company, csh_flow_df.Value.alias('amount'), csh_flow_df.Date)

StatementMeta(, 763ba16a-a488-49ce-96cd-40c08b0e58e2, 13, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [10]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        delta_table.alias("target").merge(
            output_df.alias("source"),
                    (col("target.Breakdown") == col("source.Breakdown")) &
                    (col("target.Company") == col("source.Company")) &
                    (col("target.Date") == col("source.Date"))
        ).whenMatchedUpdateAll(
        ).whenNotMatchedInsertAll().execute()

    else:
        output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, 763ba16a-a488-49ce-96cd-40c08b0e58e2, 14, Finished, Available, Finished)